<a href="https://colab.research.google.com/github/rubuntu/Taller_Introduccion_a_Ciencia_de_Datos_IA_e_Ingenieria_de_Datos/blob/main/sesion_17_escalando_el_prototipo_con_llamaindex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Sesión 17 - Escalando el prototipo con LlamaIndex + Modelos Abiertos Cuantizados

## 🎯 Objetivos

- Introducir **LlamaIndex** como framework para construir RAGs más robustos.  
- Aprender cómo indexar datasets completos y ejecutar consultas.  
- Explorar el uso de **modelos de pesos abiertos cuantizados**.
- Evaluar la recuperación con métricas básicas y probar calidad de respuestas.  

---

## 1. 🚀 Instalación de dependencias

In [ ]:
%%capture
# Instalación de librerías necesarias en Colab
!pip install transformers accelerate bitsandbytes
!pip install llama-index-core llama-index-embeddings-huggingface llama-index-llms-huggingface
!pip install wikipedia-api
!pip install chromadb==0.4.24
!pip install llama-index-vector-stores-chroma



Antes de empezar, necesitamos instalar las librerías clave:

* `transformers`: para cargar y usar el modelo Mistral.
* `bitsandbytes`: nos permite usar cuantización en 4 bits y ahorrar memoria en GPU.
* `accelerate`: maneja la asignación automática de dispositivos (CPU/GPU).
* `llama-index`: framework para crear RAGs (Retrieval-Augmented Generation).
* `chromadb`: motor vectorial que servirá como base de datos semántica.
* `wikipedia-api`: descargará textos de Wikipedia en español.

---

## 2. 📥 Carga del modelo Mistral

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

# 📥 Descargamos el modelo Mistral desde Hugging Face
model_id = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"

# Tokenizador
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Cargar modelo en GPU
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
)

Aquí descargamos el modelo **Mistral-7B-Instruct-v0.3** de Hugging Face y lo cargamos en memoria.

* `AutoTokenizer` prepara el texto para que el modelo lo entienda.
* `device_map="auto"` hace que se cargue en GPU automáticamente si está disponible (ej. T4 en Colab).

---

## 3. 🔌 Conexión con LlamaIndex

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

# Crear LLM
llm = HuggingFaceLLM(model=model, tokenizer=tokenizer)

# Configurar como LLM por defecto
Settings.llm = llm


* `HuggingFaceLLM` envuelve el modelo de Hugging Face para que LlamaIndex pueda usarlo.
* `Settings.llm` = llm indica que todas las operaciones de LlamaIndex usarán Mistral como motor LLM.

---

## 4. ✅ Prueba rápida del modelo

In [ ]:
response = llm.complete("Cuentame de Paraguay.")
print(response)

Aquí hacemos una prueba inicial para verificar que Mistral está funcionando y que responde en **español**.

---

## 5. 📚 Descarga de documentos de Wikipedia (Paraguay)

In [ ]:
import wikipediaapi

# Configuración de Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(
    language='es',
    user_agent='MiProyectoRAG/1.0 (https://github.com/rubuntu)'
)

# Lista de páginas a descargar
paginas = [
    "Paraguay",
    "Geografía de Paraguay",
    "Economía de Paraguay",
    "Cultura de Paraguay",
    "Historia de Paraguay",
    "Asunción",
    "Guerra de la Triple Alianza",
    "Guerra del Chaco",
]

# Descargar y almacenar textos
docs_texts = []
for titulo in paginas:
    page = wiki_wiki.page(titulo)
    if page.exists():
        docs_texts.append(page.text)
        print(f"Cargado: {titulo}")
print(f"Total documentos cargados: {len(docs_texts)}")

Aquí usamos la API de Wikipedia para descargar múltiples artículos relacionados con **Paraguay**.

* Se definen las páginas de interés (historia, geografía, guerras, cultura, etc.).
* Cada artículo se descarga en español y se guarda en `docs_texts`.
* Así construimos nuestro **corpus de conocimiento**.

---

## 6. 📂 Indexación de documentos

In [ ]:
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
import chromadb

# 1. Configurar LLM
llm = HuggingFaceLLM(model=model, tokenizer=tokenizer)
Settings.llm = llm

# 2. Configurar embeddings
embed_model = HuggingFaceEmbedding(model_name="intfloat/multilingual-e5-base")
Settings.embed_model = embed_model

# 3. Documentos
documents = [Document(text=doc) for doc in docs_texts]

# 4. Dividir en nodos
splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=200)
nodes = splitter.get_nodes_from_documents(documents)

# 5. Cliente Chroma
chroma_client = chromadb.EphemeralClient()
# OR: chroma_client = chromadb.PersistentClient(path="./chroma_db")

# 6. Create collection and vector store
chroma_collection = chroma_client.get_or_create_collection(name="paraguay")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 7. Crear índice
index = VectorStoreIndex(nodes, vector_store=vector_store)

print("✅ Índice creado con colección 'paraguay'")

# 8. Consulta de prueba
query_engine = index.as_query_engine()
response = query_engine.query("¿Cuál es la capital de Paraguay?")
print("🔎 Respuesta:", response)




* Settings.llm = llm → define que todas las consultas usen Mistral como generador.
* Settings.embed_model = embed_model → define que los documentos se indexen con embeddings de Hugging Face en lugar de OpenAI.
* Elegimos intfloat/multilingual-e5-base, que funciona muy bien en español.
* Transformamos cada texto en un `Document` de LlamaIndex.
* Inicializamos Chroma como nuestra base vectorial.
* Construimos un **índice semántico** para consultas rápidas y contextuales.

---

## 7. 🔎 Consultas al índice

In [ ]:
system_prompt = """Eres un asistente experto en historia, geografía y cultura de Paraguay.
Debes responder SIEMPRE en español, de forma clara y concisa.
Si no sabes la respuesta, admite que no lo sabes.
"""

query_engine = index.as_query_engine(
    system_prompt=system_prompt
)

# Ejemplo de consultas
preguntas = [
    "¿Cuáles son los principales ríos de Paraguay?",
    "¿Qué importancia tiene la ciudad de Asunción?",
    "¿Qué países participaron en la Guerra de la Triple Alianza?",
    "Resume la economía de Paraguay en pocas frases"
]

for q in preguntas:
    response = query_engine.query(q)
    print("Pregunta:", q)
    print("Respuesta:", response, "\n")

* `system_prompt` define las instrucciones iniciales para el modelo. Aquí le decimos explícitamente: “Responde siempre en español”. Esto asegura consistencia incluso si alguna pregunta llega o se genera una respuesta en inglés
* `index.as_query_engine()` crea un motor de consulta listo para preguntas en lenguaje natural.
* Probamos con preguntas típicas sobre geografía, historia y economía de Paraguay.
* El modelo recupera fragmentos relevantes y responde en español.

---

## 7. 🧪 Evaluaciones básicas



In [ ]:
def preparar_queries_multi_chunk(index, queries_texto):
    queries_final = []
    for q in queries_texto:
        expected_ids = []
        for ref in q["reference"]:
            for node in index.docstore.docs.values():
                if ref.lower() in node.text.lower():
                    expected_ids.append(node.node_id)
        expected_ids = list(set(expected_ids))  # quitar duplicados
        queries_final.append({
            "query": q["query"],
            "expected_ids": expected_ids,
            "reference": q["reference"]
        })
    return queries_final


# ------------------------------
# Dataset en texto
# ------------------------------
queries_texto = [
    {"query": "¿Cuál es la capital de Paraguay?", "reference": ["Asunción"]},
    {"query": "¿Qué guerra enfrentó Paraguay contra Bolivia?", "reference": ["Guerra del Chaco"]},
]

queries = preparar_queries_multi_chunk(index, queries_texto)
print("Queries preparadas (multi-chunk):")
for q in queries:
    print(q)


# ------------------------------
# Retriever con top_k alto
# ------------------------------
from llama_index.core.evaluation import RetrieverEvaluator

retriever = index.as_retriever(similarity_top_k=200)
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["hit_rate", "mrr"], retriever=retriever
)


# ------------------------------
# Evaluación sin reranker
# ------------------------------
results = []
for q in queries:
    res = retriever_evaluator.evaluate(
        query=q["query"],
        expected_ids=q["expected_ids"]
    )
    results.append(res)

print("\n=== Resultados sin reranker ===")
for r in results:
    print("Query:", r.query)
    print("Expected IDs:", len(r.expected_ids))
    print("Retrieved IDs (top 5):", r.retrieved_ids[:5])
    print("Metrics:", {k: v.score for k, v in r.metric_dict.items()})
    print("—")

# Métricas agregadas
hit_rate_avg = sum(r.metric_dict["hit_rate"].score for r in results) / len(results)
mrr_avg = sum(r.metric_dict["mrr"].score for r in results) / len(results)

print(f"\nHit Rate promedio: {hit_rate_avg:.2f}")
print(f"MRR promedio: {mrr_avg:.2f}")


# ------------------------------
# Recall@k
# ------------------------------
def compute_recall_at_k(results, k=3):
    recalls = []
    for r in results:
        expected = set(r.expected_ids)
        retrieved_topk = set(r.retrieved_ids[:k])
        intersect = expected.intersection(retrieved_topk)
        recall = len(intersect) / len(expected) if expected else 0.0
        recalls.append(recall)
    return sum(recalls) / len(recalls)

for k in [3, 5, 10, 20]:
    recall_at_k = compute_recall_at_k(results, k=k)
    print(f"Recall@{k} promedio: {recall_at_k:.2f}")


# ------------------------------
# Añadir Re-ranker
# ------------------------------
from llama_index.core.postprocessor import SentenceTransformerRerank

reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-6-v2",  # rápido y liviano
    top_n=10
)

from llama_index.core import QueryBundle

print("\n=== Resultados con reranker ===")
for q in queries:
    # Recuperar candidatos
    retrieved_nodes = retriever.retrieve(q["query"])
    # Re-rankear (ahora con QueryBundle)
    query_bundle = QueryBundle(q["query"])
    reranked_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    reranked_ids = [n.node_id for n in reranked_nodes]

    expected = set(q["expected_ids"])
    retrieved_topk = set(reranked_ids[:10])
    intersect = expected.intersection(retrieved_topk)
    recall = len(intersect) / len(expected) if expected else 0.0

    print("Query:", q["query"])
    print("Top 5 IDs re-rankeados:", reranked_ids[:5])
    print("Recall@10 (con reranker):", round(recall, 2))
    print("—")



* Definimos un `retriever` que devuelve los 3 documentos más similares.
* Usamos `RetrieverEvaluator` para obtener metricas.
* Creamos un dataset de prueba con preguntas y respuestas esperadas.
* Así validamos si el sistema recupera información relevante.

---

### Calidad de respuesta

In [ ]:
query = "Resume brevemente la historia de Paraguay."
response = query_engine.query(query)
print("Respuesta generada:", response)

Aquí evaluamos la **calidad de la respuesta generada**.
El modelo debe devolver un resumen coherente y en español sobre la historia de Paraguay.

---


## 8. ❓ Preguntas de discusión

### 1. ¿Qué ventajas aporta LlamaIndex frente a implementar embeddings + vector DB manualmente?

- **Abstracción de complejidad**: en lugar de escribir decenas de líneas de código para manejar embeddings, base vectorial, consulta y recuperación, LlamaIndex lo encapsula en APIs de alto nivel como `VectorStoreIndex` y `QueryEngine`.  
- **Integración con múltiples backends**: soporta distintos motores de embeddings, bases vectoriales (Chroma, Weaviate, Pinecone, FAISS, etc.) y LLMs, sin tener que reescribir lógica de integración.  
- **Extensibilidad**: provee componentes modulares (retrievers, evaluadores, response synthesizers) que se pueden personalizar.  
- **Productividad**: permite centrarse en la lógica de negocio y evaluación de resultados, en lugar de la infraestructura.  
- **Evaluación integrada**: ya incorpora utilidades para medir "hit_rate", "mrr".  

En resumen: **menos código, más rapidez y flexibilidad**.

---

### 2. ¿Cómo se podría evaluar la calidad de las respuestas en un RAG real?

La evaluación en un sistema RAG (Retrieval-Augmented Generation) debe considerar dos niveles:

1. **Calidad de recuperación**:  
   - Métricas clásicas de IR (Information Retrieval):  
     - *Recall@k* → ¿cuántas de las respuestas correctas se encuentran en el top-k recuperado?  
     - *Precision@k* → ¿qué proporción de los documentos recuperados son relevantes?  
     - *MRR* (Mean Reciprocal Rank) → mide la posición del primer documento relevante.  
   - Estas métricas permiten cuantificar si el sistema realmente trae el contexto adecuado.  

2. **Calidad de la respuesta generada por el LLM**:  
   - **Automática**: comparar con respuestas de referencia (*exact match*, *BLEU*, *ROUGE*, *BERTScore*).  
   - **LLM-as-a-judge**: usar otro LLM para evaluar criterios como coherencia, cobertura y exactitud.  
   - **Humana**: encuestas de usuarios, validación por expertos del dominio.  
   - **Evaluación factual**: detectar alucinaciones y verificar citas contra fuentes.  

En la práctica, se combinan métricas automáticas + validación humana para un panorama más realista.

---

### 3. ¿En qué casos NO usarías RAG?

Aunque poderoso, **RAG no siempre es la mejor solución**. Ejemplos de casos donde no conviene:

- **Datos muy estructurados y pequeños**: si la información ya está en una base de datos relacional bien diseñada, una simple consulta SQL es más eficiente y precisa.  
- **Consultas determinísticas**: cuando las respuestas deben ser exactas, como en sistemas bancarios o médicos críticos, donde no se puede tolerar “alucinaciones” del modelo.  
- **Dominio cerrado con vocabulario limitado**: si el conocimiento se puede representar en reglas o tablas, un sistema simbólico puede ser más adecuado.  
- **Restricciones de costo o latencia**: RAG introduce pasos adicionales (generación de embeddings, búsqueda en vector DB, inferencia de LLM), lo que puede aumentar el tiempo de respuesta y consumo de recursos.  
- **Confidencialidad extrema**: cuando no se puede permitir almacenar o indexar información sensible, ni siquiera en bases vectoriales privadas.  

👉 En resumen: **RAG es ideal cuando tenemos grandes corpus no estructurados y queremos respuestas en lenguaje natural**, pero no siempre es la herramienta más eficiente o segura para todo escenario.

---